In [ ]:


import numpy as np 
import pandas as pd




import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
import xlrd

workbook = xlrd.open_workbook("/kaggle/input/polycystic-ovary-syndrome-pcos/PCOS_data_without_infertility.xlsx")
sheet = workbook.sheet_by_index(0)

for rowx in range(sheet.nrows):
    values = sheet.row_values(rowx)
    print(values)

In [ ]:
PCOS_inf = pd.read_csv("../input/polycystic-ovary-syndrome-pcos/PCOS_infertility.csv")
PCOS_data = pd.read_excel("../input/polycystic-ovary-syndrome-pcos/PCOS_data_without_infertility.xlsx", sheet_name='Full_new')

In [ ]:
PCOS_data.head().T


In [ ]:
PCOS_data[~ PCOS_data['Unnamed: 44'].isna()].T


In [ ]:
PCOS_data.info()

In [ ]:
PCOS_data[PCOS_data['Marraige Status (Yrs)'].isnull()].T
PCOS_data[PCOS_data['Fast food (Y/N)'].isnull()].T



In [ ]:
PCOS_data['Marraige Status (Yrs)'].fillna(PCOS_data['Marraige Status (Yrs)'].median(),inplace=True)
PCOS_data['Fast food (Y/N)'].fillna(PCOS_data['Fast food (Y/N)'].median(),inplace=True)


In [ ]:
PCOS_data.drop('Unnamed: 42',axis=1,inplace=True)


In [ ]:
PCOS_data.head(4)


In [ ]:
PCOS_inf.head(4)

In [ ]:
PCOS_inf.info()

In [ ]:
data = pd.merge(PCOS_data,PCOS_inf, on='Patient File No.', suffixes={'','_y'},how='left')

In [ ]:
data.columns = ['SNo', 'Patient_File_No.', 'PCOS_(Y/N)', 'Age_(yrs)', 'Weight_(Kg)',
       'Height(Cm)', 'BMI', 'Blood_Group', 'Pulse_rate(bpm)',
       'RR_(breaths/min)', 'Hb(g/dl)', 'Cycle(R/I)', 'Cycle_length(days)',
       'Marriage_Status_(Yrs)', 'Pregnant(Y/N)', 'No_of_aborptions',
       'FSH(mIU/mL)', 'LH(mIU/mL)', 'FSH/LH', 'Hip(inch)', 'Waist(inch)',
       'Waist:Hip_Ratio', 'TSH_(mIU/L)', 'AMH(ng/mL)', 'PRL(ng/mL)',
       'Vit_D3_(ng/mL)', 'PRG(ng/mL)', 'RBS(mg/dl)', 'Weight_gain(Y/N)',
       'hair_growth(Y/N)', 'Skin_darkening (Y/N)', 'Hair_loss(Y/N)',
       'Pimples(Y/N)', 'Fast_food_(Y/N)', 'Reg_Exercise(Y/N)',
       'BP_Systolic(mmHg)', 'BP_Diastolic(mmHg)', 'Follicle_No.(L)',
       'Follicle_No.(R)', 'Avg.Fsize(L)(mm)', 'Avg.Fsize(R)(mm)',
       'Endometrium(mm)', 'Sl.No_y', 'PCOS(Y/N)_y',
       'I_beta-HCG(mIU/mL)', 'II_beta-HCG(mIU/mL)', 'AMH(ng/mL)_y']

In [ ]:
data.drop(['Sl.No_y', 'PCOS(Y/N)_y','AMH(ng/mL)_y'],axis=1,inplace=True)

In [ ]:
data.info()

In [ ]:
data.describe().T


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
target = data['PCOS (Y/N)']
data.drop('PCOS (Y/N)',axis=1,inplace=True)

In [ ]:
plt.figure(figsize=(8,7))
sns.countplot(target)
plt.title('Data imbalance')
plt.show()

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(data, target, test_size=0.15, random_state=1, stratify = target)
X_train,X_valid, y_train, y_valid =  train_test_split(X_train, y_train, test_size=0.3, random_state=1, stratify=y_train)


In [ ]:
from sklearn.metrics import roc_auc_score
def print_scores(m):
    res = [roc_auc_score(y_train,m.predict_proba(X_train)[:,1]),roc_auc_score(y_valid,m.predict_proba(X_valid)[:,1])]
    for r in res:
        print(r)

In [ ]:
rf = RandomForestClassifier(n_jobs=-1,n_estimators=150,max_features='sqrt',min_samples_leaf=10)
rf.fit(X_train,y_train)
print_scores(rf)

In [ ]:
from sklearn.metrics import roc_curve
y_pred_proba = rf.predict_proba(X_valid)[:,1]
fpr, tpr, thresholds = roc_curve(y_valid, y_pred_proba)

In [ ]:
plt.figure(figsize=(8,7))
plt.plot([0,1],[0,1],'k--')
plt.plot(fpr,tpr, label='Knn')
plt.xlabel('fpr')
plt.ylabel('tpr')
plt.title('Knn(n_neighbors=11) ROC curve')
plt.show()

In [ ]:
def get_fi(m, df):
    return pd.DataFrame({'col': df.columns, 'imp': m.feature_importances_}).sort_values('imp',ascending=False)

#lets get the feature importances for training set
fi = get_fi(rf,X_train)

In [ ]:
def plot_fi(df):
    df.plot('col','imp','barh',figsize=(10,10))
    
plot_fi(fi)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.offline as py
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go


In [ ]:
plt.rcParams['figure.figsize'] = (100, 80)
plt.style.use('ggplot')

sns.heatmap(data.corr(), annot = True, cmap = 'Wistia')
plt.title('Heatmap for the Dataset', fontsize = 20)
plt.show()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (15, 5)
sns.distplot(data['Follicle_No.(L)'], color = 'cyan')
plt.title('Distribution of Follicle_No.(L)', fontsize = 20)
plt.show()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (15, 5)
sns.distplot(data['Follicle_No.(R)'], color = 'red')
plt.title('Distribution of Follicle_No.(R)', fontsize = 20)
plt.show()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (15, 5)
sns.distplot(data['Cycle(R/I)'], color = 'cyan')
plt.title('Distribution of Cycle(R/I)', fontsize = 20)
plt.show()

In [ ]:
size = target.value_counts()
colors = ['lightblue', 'lightgreen']
labels = "with pcos", "without pcos"
explode = [0, 0.01]

my_circle = plt.Circle((0, 0), 0.7, color = 'white')

plt.rcParams['figure.figsize'] = (9, 9)
plt.pie(size, colors = colors, labels = labels, shadow = True, explode = explode, autopct = '%.2f%%')
plt.title('Distribution of with & without PCOS', fontsize = 20)
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.legend()
plt.show()

In [ ]:
t1=data['Skin_darkening (Y/N)']

In [ ]:
size = t1.value_counts()
colors = ['lightblue', 'lightgreen']
labels = "Y", "N"
explode = [0, 0.01]

my_circle = plt.Circle((0, 0), 0.7, color = 'white')

plt.rcParams['figure.figsize'] = (9, 9)
plt.pie(size, colors = colors, labels = labels, shadow = True, explode = explode, autopct = '%.2f%%')
plt.title('Distribution of skin darkening', fontsize = 20)
p = plt.gcf()
p.gca().add_artist(my_circle)
plt.legend()
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (15, 7)
plt.style.use('seaborn-talk')
sns.countplot(target, palette = 'pastel')
plt.title('Distribution of Target', fontsize = 20)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (12, 9)
sns.boxplot(target, data['Follicle_No.(R)'], palette = 'viridis')
plt.title('Relation of Follicle_No.(R) with PCOS (Y/N)', fontsize = 20)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (12, 9)
sns.boxplot(target, data['Follicle_No.(L)'], palette = 'viridis')
plt.title('Relation of Follicle_No.(L) with PCOS (Y/N)', fontsize = 20)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (12, 9)
sns.violinplot(target, data['hair_growth(Y/N)'], palette = 'colorblind')
plt.title('Relation of Hair Growth with PCOS(Y/N) - Violin Plot', fontsize = 20, fontweight = 30)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (12, 9)
dat = pd.crosstab(target, data['Weight_gain(Y/N)']) 
dat.div(dat.sum(1).astype(float), axis = 0).plot(kind = 'bar', 
                                                 stacked = False, 
                                                 color = plt.cm.rainbow(np.linspace(0, 1, 4)))
plt.title('Relation of Cycle with PCOS(Y/N)', fontsize = 20, fontweight = 30)
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (15, 9)
sns.boxenplot(target, data['Follicle_No.(R)'], palette = 'copper')
plt.title('Relation between Follicle_No.(R) and PCOS(Y/N)', fontsize = 20, fontweight = 30)
plt.show()


In [ ]:
plt.rcParams['figure.figsize'] = (15, 9)
sns.boxenplot(target, data['Follicle_No.(L)'], palette = 'copper')
plt.title('Relation between Follicle_No.(L) and PCOS(Y/N)', fontsize = 20, fontweight = 30)
plt.show()


In [ ]:
data = pd.get_dummies(data, drop_first=True)


In [ ]:
data.head()

In [ ]:
y=target

In [ ]:
x = data

# checking the shapes of x and y
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

In [ ]:
y.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

# getting the shapes
print("Shape of x_train :", x_train.shape)
print("Shape of x_test :", x_test.shape)
print("Shape of y_train :", y_train.shape)
print("Shape of y_test :", y_test.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

model = RandomForestClassifier(n_estimators = 50, max_depth = 5)
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
y_pred_quant = model.predict_proba(x_test)[:, 1]
y_pred = model.predict(x_test)

print("Training Accuracy :", model.score(x_train, y_train))
print("Testing Accuracy :", model.score(x_test, y_test))

cm = confusion_matrix(y_test, y_pred)
plt.rcParams['figure.figsize'] = (5, 5)
sns.heatmap(cm, annot = True, annot_kws = {'size':15}, cmap = 'PuBu')

cr = classification_report(y_test, y_pred)
print(cr)

In [ ]:
from sklearn.tree import export_graphviz

estimator = model.estimators_[1]
feature_names = [i for i in x_train.columns]

y_train_str = y_train.astype('str')
y_train_str[y_train_str == '0'] = 'no disease'
y_train_str[y_train_str == '1'] = 'disease'
y_train_str = y_train_str.values


export_graphviz(estimator, out_file='tree.dot', 
                feature_names = feature_names,
                class_names = y_train_str,
                rounded = True, proportion = True, 
                label='root',
                precision = 2, filled = True)

from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=50'])

from IPython.display import Image
Image(filename = 'tree.png')

In [ ]:
total=sum(sum(cm))

sensitivity = cm[0,0]/(cm[0,0]+cm[1,0])
print('Sensitivity : ', sensitivity )

specificity = cm[1,1]/(cm[1,1]+cm[0,1])
print('Specificity : ', specificity)

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_test, y_pred_quant)

fig, ax = plt.subplots()
ax.plot(fpr, tpr)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="-", c=".3")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])

plt.rcParams['figure.figsize'] = (15, 5)
plt.title('ROC curve for PCOS classifier', fontweight = 30)
plt.xlabel('False Positive Rate (1 - Specificity)')
plt.ylabel('True Positive Rate (Sensitivity)')
plt.show()

In [ ]:
from sklearn.metrics import auc
auc = auc(fpr, tpr)
print("AUC Score :", auc)

In [ ]:
import eli5 
from eli5.sklearn import PermutationImportance

#for SHAP values
import shap 
from pdpbox import pdp, info_plots

In [ ]:
perm = PermutationImportance(model, random_state = 0).fit(x_test, y_test)
eli5.show_weights(perm, feature_names = x_test.columns.tolist())

In [ ]:
base_features = data.columns.values.tolist()

feat_name = 'Follicle_No.(L)'
pdp_dist = pdp.pdp_isolate(model=model, dataset=x_test, model_features = base_features, feature = feat_name)

pdp.pdp_plot(pdp_dist, feat_name)
plt.show()


In [ ]:
base_features = data.columns.values.tolist()

feat_name = 'Follicle_No.(R)'
pdp_dist = pdp.pdp_isolate(model=model, dataset=x_test, model_features = base_features, feature = feat_name)

pdp.pdp_plot(pdp_dist, feat_name)
plt.show()

In [ ]:
base_features = data.columns.values.tolist()

feat_name = 'Fast_food_(Y/N)'
pdp_dist = pdp.pdp_isolate(model=model, dataset=x_test, model_features = base_features, feature = feat_name)

pdp.pdp_plot(pdp_dist, feat_name)
plt.show()

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(x_test)

shap.summary_plot(shap_values[1], x_test, plot_type="bar")

In [ ]:
def patient_analysis(model, patient):
  explainer = shap.TreeExplainer(model)
  shap_values = explainer.shap_values(patient)
  shap.initjs()
  return shap.force_plot(explainer.expected_value[1], shap_values[1], patient)

In [ ]:
patients = x_test.iloc[1,:].astype(float)
patient_analysis(model, patients)

In [ ]:
shap.dependence_plot('Follicle_No.(R)', shap_values[1], x_test, interaction_index = 'Follicle_No.(L)')

In [ ]:
shap_values = explainer.shap_values(x_train.iloc[:50])
shap.initjs()
shap.force_plot(explainer.expected_value[1], shap_values[1], x_test.iloc[:50])

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 6,metric='minkowski')
knn.fit(x_train, y_train)
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(x_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(x_test, y_test)))



In [ ]:
from sklearn.linear_model import LogisticRegression
C_param_range = [0.001,0.01,0.1,1,10,100]
for i in C_param_range:
    model = LogisticRegression(random_state=0,C=i).fit(x_train, y_train)

    print('Accuracy of logistic regression on training set: {:.2f} for C value {:.2f}'
     .format(model.score(x_train, y_train),i))
    print('Accuracy of logistic regression on test set: {:.2f} for C value{:.2f}'
     .format(model.score(x_test, y_test),i))


In [ ]:
import pandas
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
models.append(('LR', LogisticRegression(C=10000)))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier(n_neighbors=2)))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma=0.0001,C=1000)))
models.append(('RFC',RandomForestClassifier(n_jobs=-1,n_estimators=150,max_features='sqrt',min_samples_leaf=10)))
models.append(('RC',RidgeClassifier()))

# evaluate each model in turn
results = []
names = []

scoring = 'accuracy'
for name, model1 in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model1, x_train, y_train, cv=10, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)

ax.set_xticklabels(names)
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

model = RandomForestClassifier(n_estimators = 50, max_depth = 5)
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
y_pred_quant = model.predict_proba(x_test)[:, 1]
y_pred = model.predict(x_test)

print("Training Accuracy :", model.score(x_train, y_train))
print("Testing Accuracy :", model.score(x_test, y_test))

cm = confusion_matrix(y_test, y_pred)
plt.rcParams['figure.figsize'] = (5, 5)
sns.heatmap(cm, annot = True, annot_kws = {'size':15}, cmap = 'PuBu')

cr = classification_report(y_test, y_pred)
print(cr)

In [ ]:
from sklearn.linear_model import RidgeClassifier

from yellowbrick.classifier import PrecisionRecallCurve
viz = PrecisionRecallCurve(SVC())
viz.fit(x_train, y_train)
print(viz)
viz.score(x_test, y_test)
viz.show()

In [ ]:
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.pipeline import Pipeline
nca = NeighborhoodComponentsAnalysis(random_state=4)
knn = KNeighborsClassifier(n_neighbors=3)
nca_pipe = Pipeline([('nca', nca), ('knn', knn)])
nca_pipe.fit(x_train, y_train)
print(nca_pipe.score(x_test, y_test))

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2).fit_transform(x_train)
plt.figure(dpi=120)
plt.scatter(pca[y_train.values==0,0], pca[y_train.values==0,1], alpha=0.5, label='Fertile', s=2)
plt.scatter(pca[y_train.values==1,0], pca[y_train.values==1,1], alpha=0.5, label='Infertile', s=2)
plt.legend()
plt.gca().set_aspect('equal')

In [ ]:
data

In [ ]:
data = pd.merge(PCOS_data,PCOS_inf, on='Patient File No.', suffixes={'','_y'},how='left')
data.columns = ['SNo', 'Patient_File_No.', 'PCOS_(Y/N)', 'Age_(yrs)', 'Weight_(Kg)',
       'Height(Cm)', 'BMI', 'Blood_Group', 'Pulse_rate(bpm)',
       'RR_(breaths/min)', 'Hb(g/dl)', 'Cycle(R/I)', 'Cycle_length(days)',
       'Marriage_Status_(Yrs)', 'Pregnant(Y/N)', 'No_of_aborptions',
       'FSH(mIU/mL)', 'LH(mIU/mL)', 'FSH/LH', 'Hip(inch)', 'Waist(inch)',
       'Waist:Hip_Ratio', 'TSH_(mIU/L)', 'AMH(ng/mL)', 'PRL(ng/mL)',
       'Vit_D3_(ng/mL)', 'PRG(ng/mL)', 'RBS(mg/dl)', 'Weight_gain(Y/N)',
       'hair_growth(Y/N)', 'Skin_darkening (Y/N)', 'Hair_loss(Y/N)',
       'Pimples(Y/N)', 'Fast_food_(Y/N)', 'Reg_Exercise(Y/N)',
       'BP_Systolic(mmHg)', 'BP_Diastolic(mmHg)', 'Follicle_No.(L)',
       'Follicle_No.(R)', 'Avg.Fsize(L)(mm)', 'Avg.Fsize(R)(mm)',
       'Endometrium(mm)', 'Sl.No_y', 'PCOS(Y/N)_y',
       'I_beta-HCG(mIU/mL)', 'II_beta-HCG(mIU/mL)', 'AMH(ng/mL)_y']
data.drop(['Sl.No_y', 'PCOS(Y/N)_y','AMH(ng/mL)_y'],axis=1,inplace=True)
target = data['PCOS_(Y/N)']
data.drop('PCOS_(Y/N)',axis=1,inplace=True)


In [ ]:
data.loc[data['AMH(ng/mL)']=='a']

In [ ]:
data.at[305,'AMH(ng/mL)']=np.nan
data['AMH(ng/mL)'].fillna(data['AMH(ng/mL)'].median(),inplace=True)


In [ ]:
data.iloc[305,:]

In [ ]:
data[data['AMH(ng/mL)'].isnull()].T

In [ ]:
data.iloc[305,:]

In [ ]:
data1=data[['BMI','Weight_(Kg)','Age_(yrs)','Reg_Exercise(Y/N)','Pimples(Y/N)','Hip(inch)', 'Waist(inch)','hair_growth(Y/N)', 'Skin_darkening (Y/N)', 'Hair_loss(Y/N)','BP_Systolic(mmHg)', 'BP_Diastolic(mmHg)','Cycle(R/I)', 'Cycle_length(days)','Pregnant(Y/N)', 'No_of_aborptions']]

In [ ]:
data2=data[['Age_(yrs)', 'Weight_(Kg)',
       'Height(Cm)', 'BMI', 'Blood_Group', 'Pulse_rate(bpm)',
       'RR_(breaths/min)', 'Hb(g/dl)', 'Cycle(R/I)', 'Cycle_length(days)',
       'Marriage_Status_(Yrs)', 'Pregnant(Y/N)', 'No_of_aborptions',
       'FSH(mIU/mL)', 'LH(mIU/mL)', 'FSH/LH', 'Hip(inch)', 'Waist(inch)',
       'Waist:Hip_Ratio', 'TSH_(mIU/L)', 'AMH(ng/mL)', 'PRL(ng/mL)',
       'Vit_D3_(ng/mL)', 'PRG(ng/mL)', 'RBS(mg/dl)', 'Weight_gain(Y/N)',
       'hair_growth(Y/N)', 'Skin_darkening (Y/N)', 'Hair_loss(Y/N)',
       'Pimples(Y/N)', 'Fast_food_(Y/N)', 'Reg_Exercise(Y/N)',
       'BP_Systolic(mmHg)', 'BP_Diastolic(mmHg)', 'Follicle_No.(L)',
       'Follicle_No.(R)', 'Avg.Fsize(L)(mm)', 'Avg.Fsize(R)(mm)',
       'Endometrium(mm)',
       'I_beta-HCG(mIU/mL)', 'II_beta-HCG(mIU/mL)']]

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data,target, test_size = 0.2, random_state = 0)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import LinearSVC 
model1 = LinearSVC() 
model1.fit(x_train, y_train) 
  
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}  
  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
  
# fitting the model for grid search 
grid.fit(x_train, y_train) 



In [ ]:

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import svm
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.gaussian_process.kernels import RBF
from sklearn.linear_model import RidgeClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
classifier = RandomForestClassifier(max_depth=2300,n_estimators=9000, random_state=0, max_features=0.3,min_samples_leaf=0.001)
classifier10 = RandomForestClassifier(max_depth=600,n_jobs=-1,n_estimators=1000,max_features=0.3,min_samples_leaf=10)
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

classifier = KNeighborsClassifier(n_neighbors = 6,metric='minkowski')
classifier1=RidgeClassifier(alpha=10.0,solver='cholesky')
classifier2 =  AdaBoostClassifier(n_estimators=600, learning_rate=0.01, algorithm='SAMME')
classifier3=QuadraticDiscriminantAnalysis()
classifier4= LinearDiscriminantAnalysis()
classifier5=GaussianNB()
calssifier6=GaussianProcessClassifier()
classifier7=GradientBoostingClassifier(n_estimators=90000, learning_rate=0.001, max_features=30, random_state=0)
classifier8=LogisticRegression(C=0.1,penalty='l1',solver= 'liblinear', max_iter=100)
classifier9=XGBClassifier()
classifier5=svm.SVC()
#classifier=MLPClassifier(alpha=0.001, max_iter=1000)
classifier.fit(x_train, y_train) 

y_pred = classifier.predict(x_test) 

from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred) 
  
print ("Confusion Matrix : \n", cm) 

from sklearn.metrics import accuracy_score 
print ("test accuracy : ", accuracy_score(y_test, y_pred)) 





In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test,y_pred)

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
print(precision_score(y_test, y_pred, average='macro'))
print(precision_score(y_test, y_pred, average='micro'))
print(precision_score(y_test, y_pred, average='weighted'))
print(recall_score(y_test,y_pred, average='macro'))
print(recall_score(y_test,y_pred, average='micro'))
print(recall_score(y_test,y_pred, average='weighted'))
print(f1_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='micro'))
print(f1_score(y_test, y_pred, average='weighted'))

In [ ]:
from sklearn.tree import export_graphviz

estimator = classifier.estimators_[1]
feature_names = [i for i in x_train.columns]

y_train_str = y_train.astype('str')
y_train_str[y_train_str == '0'] = 'no disease'
y_train_str[y_train_str == '1'] = 'disease'
y_train_str = y_train_str.values


export_graphviz(estimator, out_file='tree.dot', 
                feature_names = feature_names,
                class_names = y_train_str,
                rounded = True, proportion = True, 
                label='root',
                precision = 2, filled = True)

from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=50'])

from IPython.display import Image
Image(filename = 'tree.png')

In [ ]:
x_test.describe()

In [ ]:
predictions = classifier.predict(x_test) 
print(classification_report(y_test, predictions))

In [ ]:
from yellowbrick.classifier import PrecisionRecallCurve
viz = PrecisionRecallCurve(classifier)
viz.fit(x_train, y_train)
print(viz)
viz.score(x_test, y_test)
viz.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn import linear_model
from scipy.special import expit
clf = linear_model.LogisticRegression(C=1e5)
clf.fit(x_train, y_train)

plt.figure(1, figsize=(4, 3))
plt.clf()
plt.scatter(x_train.ravel(), y, color='black', zorder=20)


loss = expit(x_test * clf.coef_ + clf.intercept_).ravel()
plt.plot(x_test, loss, color='red', linewidth=3)


plt.ylabel('y')
plt.xlabel('X')
plt.xticks(range(-5, 10))
plt.yticks([0, 0.5, 1])
plt.ylim(-.25, 1.25)
plt.xlim(-4, 10)
plt.legend(('Logistic Regression Model', 'Linear Regression Model'),
           loc="lower right", fontsize='small')
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,y_pred)
total=sum(sum(cm))

sensitivity = cm[0,0]/(cm[0,0]+cm[1,0])
print('Sensitivity : ', sensitivity )

specificity = cm[1,1]/(cm[1,1]+cm[0,1])
print('Specificity : ', specificity)

In [ ]:
x_train.head(4)


In [ ]:
y_train.head(4)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
roc_auc
max_depths = np.linspace(1000, 10000, 9, endpoint=True)
train_results = []
test_results = []
for max_depth in max_depths:
   rf = RandomForestClassifier(max_depth=max_depth, n_jobs=4)
   rf.fit(x_train, y_train)
   train_pred = rf.predict(x_train)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   train_results.append(roc_auc)
   y_pred = rf.predict(x_test)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   test_results.append(roc_auc)
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(max_depths, train_results, 'b', label="Train AUC")
line2, = plt.plot(max_depths, test_results, 'r', label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('Tree depth')
plt.show()

In [ ]:
n_estimators = [1, 2, 4, 8, 16, 32, 64, 1000, 2000,9000]
train_results = []
test_results = []
for estimator in n_estimators:
   rf = RandomForestClassifier(n_estimators=estimator, n_jobs=-1)
   rf.fit(x_train, y_train)
   train_pred = rf.predict(x_train)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   train_results.append(roc_auc)
   y_pred = rf.predict(x_test)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   test_results.append(roc_auc)
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(n_estimators, train_results, 'b', label="Train AUC")
line2, = plt.plot(n_estimators, test_results, 'r', label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('n_estimators')
plt.show()

In [ ]:
max_features = list(range(1,x_train.shape[1]))
train_results = []
test_results = []
for max_feature in max_features:
   rf = RandomForestClassifier(max_features=max_feature)
   rf.fit(x_train, y_train)
   train_pred = rf.predict(x_train)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   train_results.append(roc_auc)
   y_pred = rf.predict(x_test)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   test_results.append(roc_auc)
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(max_features, train_results, 'b', label="Train AUC")
line2, = plt.plot(max_features, test_results, 'r', label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('max_features')
plt.show()

In [ ]:
y_pred_quant = classifier.predict_proba(x_test)[:, 1]
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_test, y_pred_quant)

fig, ax = plt.subplots()
ax.plot(fpr, tpr)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="-", c=".3")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])

plt.rcParams['figure.figsize'] = (15, 5)
plt.title('ROC curve for PCOS classifier using Random Forst Classifier', fontweight = 30)
plt.xlabel('False Positive Rate(1 - Specificity)')
plt.ylabel('True Positive Rate Recall(Sensitivity)')
plt.show()
from sklearn.metrics import auc
auc = auc(fpr, tpr)
print("AUC Score :", auc)

In [ ]:
min_samples_leafs = np.linspace(0.001, 0.5, 5, endpoint=True)
train_results = []
test_results = []
for min_samples_leaf in min_samples_leafs:
   rf = RandomForestClassifier(min_samples_leaf=min_samples_leaf,random_state=0, n_jobs=-1)
   rf.fit(x_train, y_train)
   train_pred = rf.predict(x_train)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train, train_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   train_results.append(roc_auc)
   y_pred = rf.predict(x_test)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   test_results.append(roc_auc)
from matplotlib.legend_handler import HandlerLine2D
line1, = plt.plot(min_samples_leafs, train_results, 'b', label="Train AUC")
line2, = plt.plot(min_samples_leafs, test_results, 'r', label="Test AUC")
plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
plt.ylabel('AUC score')
plt.xlabel('min samples leaf')
plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

In [ ]:
classifier.predict([[26.00000,60.0,26,0,1,40,36,1,0,0,100,80,4,2,1,0]])

In [ ]:
classifier.predict([[474,39,50.0,154.0,21.10000,15,78,22,2,5,...,0,120,80	5	7	19.0	20.0	5.0	1.99	1.97]])

